# XGBoost regression for all sites
In this notebook, make a model that takes time and site id as the X values and predict y, the % available at the location.

The outcome is to save a model called reg_model.pkl to be used on the streamlit app

In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/denny/Documents/mids/ev_charging'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from src.data_preprocessing import  datetime_processing, userinput_processing, holiday_processing, create_all_site_x, create_all_site_y

In [3]:
df_c = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df_c = datetime_processing(df_c)
df_c = userinput_processing(df_c)
df_c = holiday_processing(df_c)


df_j = pd.read_parquet('data/sessions/jpl/').reset_index(drop=True)
df_j = datetime_processing(df_j)
df_j = userinput_processing(df_j)
df_j = holiday_processing(df_j)

df_o = pd.read_parquet('data/sessions/office001/').reset_index(drop=True)
df_o = datetime_processing(df_o)
df_o = userinput_processing(df_o)
df_o = holiday_processing(df_o)

cols = list(df_c.columns)

In [4]:
df = pd.concat([df_c[cols], df_j[cols], df_o[cols]], axis=0)

In [5]:
print(f'df is a dataframe comprised of these site ids: {list(df.siteID.unique())}')

df is a dataframe comprised of these site ids: [2, 1, 19]


In [6]:
# df.loc[:, 'connectionTime'].min().date().strftime('%Y-%m-%d')

## Make X and y
now for all sites. To do this, duplicate a lot of time entries, one for each site

In [12]:
# new functions that make X and y dataframes that contain data for all sites
X = create_all_site_x(df)
y = create_all_site_y(df, regression=True)
assert X.shape[0] == y.shape[0], 'X and y rows are not equal!'

## Modeling, XGBoost Regression

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

regression_model = Pipeline([
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')),
    ('xgb', XGBRegressor()),
])

regression_model.fit(X_train, y_train)

print(f'training loss: {regression_model.score(X_train, y_train)}\ntest loss    : {regression_model.score(X_test, y_test)}')

training loss: 0.4932504521769091
test loss    : 0.47327013819340447


# Save model as reg_model.pkl

In [9]:
import pickle
pickle.dump(regression_model, open('../reg_model.pkl','wb'))
os.getcwd()

'/home/denny/Documents/mids/ev_charging'

In [10]:
# confirm model
model = pickle.load(open('../reg_model.pkl', 'rb'))
model

Pipeline(steps=[('ohe',
                 OneHotEncoder(drop='first', handle_unknown='ignore',
                               sparse_output=False)),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])